In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from jax.scipy.stats import multivariate_normal
from statsmodels.graphics.tsaplots import plot_acf

import jax
import jax.numpy as jnp
from jax import grad, jit, vmap

# Question 1: Reading and formatting dataset

In [ ]:
#1.1 : Read the datset with pandas
dataset = pd.read_csv("GermanCredit.txt", sep ="\s+", header=None)
dataset

In [ ]:
#1.2 : Creating ytrain. For convenience, we will use 0 and 1 as labels
dataset[24] = dataset[24] - 1
dataset

In [ ]:
# 1.2 Split into train and test set
M = 800 # train set size
d = 24
length = dataset.shape[0]
y_train = dataset.loc[:M-1, d].to_numpy()
y_test = dataset.loc[M:, d].to_numpy()

x_train = dataset.loc[:M-1, :d-1]
x_test = dataset.loc[M:, :d-1]

In [ ]:
#1.3 Center and scale the features
from sklearn.preprocessing import StandardScaler

In [ ]:
#1.3 : scaling xtrain 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape

In [ ]:
#1.4 Extend them with a column of ones, which is for logistic regression
ones_train = jnp.ones((M, 1))
ones_test = jnp.ones((length-M, 1))
x_train = jnp.concatenate((ones_train, x_train), axis=1)
x_train.shape

In [ ]:
# the shape changes to dimension 25
x_test = jnp.concatenate((ones_test, x_test), axis=1)
x_test

In [ ]:
#make sure that the values are scaled 
jnp.mean(x_train, axis=0), jnp.std(x_train, axis=0)

# Question 2: Model specification

## Proof of log-odds ratio
Let us denote $$z= \beta_0 + \sum_{j = 1}^n \beta_j X_j.$$

We are to show that
$$ \log \frac{P(Y=1|\beta)}{1-P(Y=1|\beta)} = z.$$
We can rewrite the left hand side by splitting the fraction of the log
\begin{align*} 
\log \frac{P(Y=1|\beta)}{1-P(Y=1|\beta)} &=  \log \frac{1}{1+e^{-z}} - \log \frac{e^{-z}}{1+e^{-z}}\\ 
 &=   \log 1 - \log e^{-z}\\ 
 &=  - \log e^{-z}\\ 
 &= z
\end{align*}
Note that in the second step we used that you can split again to 4 logarithms and then the log of the denominators will have opposite sign.

## Interpreting the parameters
These parameters represent the importance of features $X = (X_1,...,X_d)$ when we decide how likely $Y=1$(the individual having bad credit risk) compared to $Y=0$(the individual having good credit risk).

## Decision Boundry
When $\beta_0 + \sum_{j = 1}^n \beta_j X_j$ is positive, it means the individual has a bad credit risk.Otherwise, he has a good credit risk. After fitting it into sigmoid function, the threshold will be 0.5, because $expit(0) = \log \frac{1}{1+e^{0}} = 0.5 $.

## Proof of log-likelihood
Let us denote $$z_i= \beta_0 + \sum_{j = 1}^n \beta_j X_{i,j}.$$

If $y_i = 1$, we see that the right hand side is 
\begin{align*} 
\log \frac{1}{1+e^{-z_i}}  &= \log \frac{e^z_i}{1+e^{z_i}}\\
&= z_i - \log 1+e^z_i\\
&= y_i z_i - \log 1+e^z_i.
\end{align*}

If $y_i = 0$, we see that the right hand side is 
\begin{align*} 
\log \frac{e^{-z_i}}{1+e^{-z_i}}  &= \log \frac{1}{1+e^{z_i}}\\
&= 0 - \log 1+e^{z_i}\\
&= y_i z_i - \log 1+e^{z_i}.
\end{align*}

## JAX-compatible log-likelihood

We are asked to calculate
$$\log P(y_1,...,y_m| \beta).$$

By independence of the training data points, this equals
$$\log \prod_{i=1}^m P(y_i| \beta).$$

We can now use the result from previous exercise to see that 

\begin{align*} 
\log \prod_{i=1}^m P(y_i| \beta) &=  \sum_{i=1}^m \log P(y_i| \beta) \\
&= \sum_{i=1}^m (y_i z_i - \log 1+e^z_i)\\
&= \sum_{i=1}^m y_i z_i -  \sum_{i=1}^m \log 1+e^z_i.
\end{align*}

In [ ]:
# # Write log-likelihood function in numpy
# def log_likelihood(beta):
#     x_beta = np.matmul(x_train, beta)
#     output = np.sum(y_train * x_beta - np.log(1 + np.exp(x_beta)))
#     return output


In [ ]:
# Write log-likelihood function in a JAX-compatible way
@jit
def log_likelihood_jax(beta):
    x_beta = jnp.matmul(x_train, beta)
    output = jnp.sum(y_train * x_beta - jnp.log(1 + jnp.exp(x_beta)))
    return output

In [ ]:
# To compare the speed with or without jit
import time

# simulate an array of beta
beta = np.random.rand(25)
jit_loglikelihood_jax = jit(log_likelihood_jax)
%timeit jit_loglikelihood_jax(beta)
%timeit log_likelihood_jax(beta)

## There is no significant speedup, we won't use jit in this case

## Gradient of the log likelihood

In [ ]:
gradloglikelihood1 = grad(log_likelihood_jax) # without JIT
gradloglikelihood2 = jit(grad(log_likelihood_jax)) # with JIT 

# for the gradient, there is a signfiicant speedup with JIT
%timeit gradloglikelihood1(beta)
%timeit gradloglikelihood2(beta)

# After investigation, the jit implementations speeds up significantly. Use the faster implementation
gradloglikelihood = jit(grad(log_likelihood_jax))


## Log-prior density evaluations

In [ ]:
DIM = 25
constant = np.pi**2 * M / (3*DIM)
Sigma = constant * np.linalg.inv(np.matmul(x_train.T, x_train))

@jit
def log_prior(beta):
    '''
    Input – beta: a vector of size d+1
    Output – log prior density: constant
    '''
    return multivariate_normal.logpdf(beta, mean=jnp.zeros(DIM), cov=Sigma)


# After investigation, the jit implementations does not speed up significantly

In [ ]:
log_prior1 = log_prior # without JIT
log_prior2 = jit(log_prior) # with JIT 
%timeit log_prior1(beta)
%timeit log_prior2(beta)

## Gradient of the log-prior evaluations

In [ ]:
grad_log_prior1 = grad(log_prior)
grad_log_prior2 = jit(grad(log_prior))

%timeit grad_log_prior1(beta)
%timeit grad_log_prior2(beta)

# After investigation, jit speeds up gradient computation significantly
grad_log_prior = grad_log_prior2

## Unnormalized log-posterior density

In [ ]:
def log_posterior(beta):
    '''
    Input – beta: a vector of size d+1
    Output – log posterior: constant
    '''
    return log_prior(beta) + log_likelihood_jax(beta)


## Gradient of log-posterior density

In [ ]:
def grad_log_posterior(beta):
    '''
    Input – beta: a vector of size d+1
    Output – gradient step of log posterior: beta: a vector of size d+1.
    '''
    return grad_log_prior(beta) + gradloglikelihood(beta)

# Markov chain Monte Carlo sampling

## Independent Metropolis–Hastings algorithm

In [ ]:
# since jax's random generation is painful, we will switch to standard scipy
import scipy
def sample_prior():
    return scipy.stats.multivariate_normal.rvs(mean=np.zeros(DIM), cov=Sigma)

n_accept = 0
N = 10000

# Initialize the Markov Chain
current_beta = sample_prior()
store_beta = np.zeros((N, DIM))
for n in range(N):
    # sample a proposed state
    proposed_beta = sample_prior()

    # evaluate posterior density
    log_posterior_proposed = log_posterior(proposed_beta)
    log_posterior_current = log_posterior(current_beta)

    # evaluate transition likelihood
    log_transition_proposed = log_prior(proposed_beta)
    log_transition_current = log_prior(current_beta)
    
    # log acceptance prob
    log_accept_prob = (log_posterior_proposed + log_transition_current
                       - log_posterior_current - log_transition_proposed)

    # accept or reject
    uniform = np.random.rand(1) # sample a uniform on [0,1]
    if np.log(uniform) < log_accept_prob:
        current_beta = proposed_beta.copy() #accept
        n_accept += 1
    # store states
    store_beta[n,:] = current_beta

In [ ]:
print("Acceptance rate: ", n_accept/N)

iteration = np.arange(1, N+1)
plt.figure()
plt.plot(iteration, store_beta[:,0])
plt.plot(iteration, store_beta[:,1])
plt.plot(iteration, store_beta[:,2])
plt.plot(iteration, store_beta[:,3])
plt.xlabel('iteration')
plt.ylabel('beta')
plt.show()

We see that the acceptance rate is 0.0017 which means we mostly reject the transition of beta and it explains why we tend to get stuck in the same beta shown in the graph. Our Markov Chain Monte-Carlo sampler is converging slowly.

## A random walk Metropolis–Hastings algorithm

In [ ]:
#3.2 Experiment with the choice of sigma
sigma_list = [0.002, 0.02, 0.2]
for sigma in sigma_list:
    noise_variance = np.identity(DIM) * sigma**2
    n_accept = 0
    current_beta = sample_prior()
    store_beta = np.zeros((N, DIM))

    for n in range(N):
        noise = scipy.stats.multivariate_normal.rvs(mean=np.zeros(DIM),
                                       cov=noise_variance)
        proposed_beta = current_beta + noise

        #evaluate posterior density
        log_posterior_proposed = log_posterior(proposed_beta)
        log_posterior_current = log_posterior(current_beta)

        #accept or reject
        log_accept_prob = log_posterior_proposed - log_posterior_current
        uniform = np.random.rand(1) # sample a uniform on [0,1]
        if np.log(uniform) < log_accept_prob:
            current_beta = proposed_beta.copy() #accept
            n_accept += 1

        store_beta[n,:] = current_beta

    plt.figure()
    plt.plot(iteration, store_beta[:,1])
    plt.xlabel('iteration')
    plt.ylabel('beta')
    plt.title("Evolution of beta")
    plt.show()

    plot_acf(store_beta[:,1], alpha=None)
    plt.ylim([0, 1.1])
    plt.xlabel('lag')
    plt.ylabel('autocorrelation')
    plt.show()
    print(f"Acceptance probability is {n_accept/N}.")

Based on autocorrelation graph, we can tell sigma=0.02 is best among three as it drops down to 0 faster than two others. However, it is not optimal as it's still far away from 0.

## Metropolis-adjusted Langevin algorithm

In [ ]:
sigma_list = [0.04, 0.08, 0.12]
for sigma in sigma_list:
    n_accept = 0
    current_beta = sample_prior()
    store_beta = np.zeros((N, DIM))
    noise_variance = np.identity(DIM) * sigma**2
    for n in range(N):
        noise = scipy.stats.multivariate_normal.rvs(mean=np.zeros(DIM),
                                       cov=noise_variance)
        proposed_beta = (current_beta +
                         0.5 * sigma**2 * grad_log_posterior(current_beta) +
                         noise)

        #evaluate posterior density
        log_posterior_proposed = log_posterior(proposed_beta)
        log_posterior_current = log_posterior(current_beta)
        log_transition_proposed = multivariate_normal.logpdf(
            proposed_beta,
            mean=current_beta
                 + 0.5 * sigma**2 * grad_log_posterior(current_beta),
            cov=noise_variance)
        log_transition_current = scipy.stats.multivariate_normal.logpdf(
            current_beta,
            mean=proposed_beta
                 + 0.5 * sigma**2 * grad_log_posterior(proposed_beta),
            cov=noise_variance)

        #accept or reject
        log_accept_prob = (log_posterior_proposed
                           + log_transition_current
                           - log_posterior_current
                           - log_transition_proposed)
        uniform = np.random.rand(1) # sample a uniform on [0,1]
        if np.log(uniform) < log_accept_prob:
            current_beta = proposed_beta.copy() #accept
            n_accept += 1

        store_beta[n,:] = current_beta

    plt.figure()
    plt.plot(iteration, store_beta[:,1])
    plt.xlabel('iteration')
    plt.ylabel('beta')
    plt.title("Evolution of beta")
    plt.show()

    plot_acf(store_beta[:,1], alpha=None)
    plt.ylim([0, 1.1])
    plt.xlabel('lag')
    plt.ylabel('autocorrelation')
    plt.show()
    print(f"Acceptance probability is {n_accept/N}.")

The sigma=0.08 seems to be the best choice. First and second choices have comparable performance on diagnostic plots. The trace plots of beta show stationarity while the second choice shows a faster drop on autocorrelation. Considering the balance between large move and decent acceptance ratio, sigma=0.08 performs better.

## Hamiltonian Monte Carlo algorithm

In [ ]:
#3.4 : Hamiltonian Monte Carlo algorithm
from hmc import hamiltonian_dynamics
step_size_list = [0.01, 0.05, 0.1]
number_step_list = [5,10]

for step_size in step_size_list:
    for number_step in number_step_list:
        n_accept = 0
        current_beta = sample_prior()
        store_beta = np.zeros((N, DIM))
        for n in range(N):
            velocity = scipy.stats.multivariate_normal.rvs(mean=np.zeros(DIM), cov=np.identity(DIM))
            proposed_beta, proposed_velocity = hamiltonian_dynamics(
                current_beta, velocity, step_size, number_step, grad_log_posterior)
            log_posterior_proposed = log_posterior(proposed_beta)
            log_posterior_current = log_posterior(current_beta)
            log_prob_proposed = multivariate_normal.logpdf(
                proposed_velocity, mean=np.zeros(DIM), cov=np.identity(DIM))
            log_prob_current = multivariate_normal.logpdf(
                velocity, mean=np.zeros(DIM), cov=np.identity(DIM))

            #accept or reject
            log_accept_prob = (log_posterior_proposed
                               + log_prob_proposed
                               - log_posterior_current
                               - log_prob_current)
            uniform = np.random.rand(1) # sample a uniform on [0,1]
            if np.log(uniform) < log_accept_prob:
                current_beta = proposed_beta.copy() #accept
                n_accept += 1

            store_beta[n,:] = current_beta

        plt.figure()
        plt.plot(iteration, store_beta[:,1])
        plt.xlabel('iteration')
        plt.ylabel('beta')
        plt.title("Evolution of beta")
        plt.show()

        plot_acf(store_beta[:,1], alpha=None)
        plt.ylim([0, 1.1])
        plt.xlabel('lag')
        plt.ylabel('autocorrelation')
        plt.show()
        print(f"Step size of {step_size} with {number_step} steps")
        print(f"Acceptance probability is {n_accept/N}.")

Based on diagnostic plots and acceptance ratios, step size of 0.05 with 5 steps is the best setup as the autocorrelation plumps to 0 very fast and trace plot shows quite stable stationarity. Meanwhile, 0.8513 acceptance ratio is decent and 0.05 as step size can ensure a larger move in each step.

In [ ]:
# Perform a final run of the Markov chain, with the best stepsize that you found and 10 number of steps, 
# for 11, 000 iterations and discard the first 1000 iterations as burn-in.
N = 11000
current_beta = sample_prior()
store_beta = np.zeros((10000, DIM))
step_size = 0.05
number_step = 10
for n in range(N):
    velocity = scipy.stats.multivariate_normal.rvs(mean=np.zeros(DIM), cov=np.identity(DIM))
    proposed_beta, proposed_velocity = hamiltonian_dynamics(
        current_beta, velocity, step_size, number_step, grad_log_posterior)
    log_posterior_proposed = log_posterior(proposed_beta)
    log_posterior_current = log_posterior(current_beta)
    log_prob_proposed = multivariate_normal.logpdf(
        proposed_velocity, mean=np.zeros(DIM), cov=np.identity(DIM))
    log_prob_current = multivariate_normal.logpdf(
        velocity, mean=np.zeros(DIM), cov=np.identity(DIM))

    #accept or reject
    log_accept_prob = (log_posterior_proposed
                       + log_prob_proposed
                       - log_posterior_current
                       - log_prob_current)
    uniform = np.random.rand(1) # sample a uniform on [0,1]
    if np.log(uniform) < log_accept_prob:
        current_beta = proposed_beta.copy() #accept
    if n >= 1000:
        store_beta[n-1000,:] = current_beta


# Model Predictions

## Explanations

Under the integral, we can see the expit function and a probability distribution. To estimate this integral, we sample $N$ points $\{ \beta^{(t)}, \space t \in \{1, .. , N\} \space \}$ from the distribution $P(\beta | y_1,..,y_m)$ and then add the obtained values $$\text{expit}(z^{(t)})$$ where 
$$z^{(t)}= \beta_0^{(t)} + \sum_{j = 1}^n \beta_j^{(t)} X_{j}.$$
We give every point $\beta^{(t)}$ a weight of $1/N$ so that the total weight is 1 and equals the integral of the probability function. This way, we obtain the method of the project. In summary, we use sampling to represent original distribution.

## Approximated predictive probabilities

In [ ]:
# Define sigmoid
import math

def sigmoid(x):

    if x >= 0:
        z = math.exp(-x)
        sig = 1 / (1 + z)
        return sig
    else:
        z = math.exp(x)
        sig = z / (1 + z)
        return sig

In [ ]:
y_prob_bayes = np.mean(sigmoid(store_beta @ x_test.T), axis=0)
y_prob_bayes.shape

In [ ]:
# 4.3
y_pred_bayes = np.where(y_prob_bayes < 0.5, 0, 1)
y_pred_bayes.shape

In [ ]:
#4.4 : Misclassification rate
misclassification_bayes = np.mean(y_pred_bayes != y_test)
misclassification_bayes

In [ ]:
# 4.5 : Cost function
def average_cost(y_test, y_pred):
    """
    Input – y_test: vector with good/bad credit risk
          – y_pred: pred values for good/bad credit risk
    Output – average cost: constant
    """
    cost_vector = np.zeros_like(y_test)
    cost_vector[(y_pred == 0) & (y_test == 1)] = 5
    cost_vector[(y_pred == 1) & (y_test == 0)] = 1
    return np.mean(cost_vector)

avg_cost_bayes = average_cost(y_test, y_pred_bayes)
avg_cost_bayes

In [ ]:
#4.6 : Maximum Likelihood Estimator
from scipy.optimize import minimize

current_beta = sample_prior()
criterion = log_likelihood_jax

beta_MLE = minimize(fun=lambda x: -log_likelihood_jax(x),
                    x0=current_beta,
                    jac=lambda x: -grad_log_likelihood(x)).x
beta_MLE.shape


In [ ]:
y_prob_mle = sigmoid(x_test @ beta_MLE)
y_pred_mle = np.where(y_prob_mle < 0.5, 0, 1)

In [ ]:
#4.7 : Misclassification rate
misclassification_mle = np.mean(y_pred_mle != y_test)
misclassification_mle


In [ ]:
avg_cost_mle = average_cost(y_test, y_pred_mle)
avg_cost_mle

In [ ]:
#4.8 : Prediction accuracy
(y_prob_mle -  y_prob_bayes)

f = lambda x: 5 if x ==-1 else x

error = y_pred - y_test

In [ ]:
#4.3 : Prediction rule
# if predict gives a probability smaller than 0.5, return 0,
# if it is larger than 0.5, return 1

def predict_all(x_test, store_beta):
    predictions_probs = predict_all_probs(x_test, store_beta)
    f = lambda x:  1 if (x>0.5) else 0

    solutions = [ f(x) for x in predictions_probs]
    return solutions

In [ ]:
y_pred = predict_all(x_test, store_beta)
y_pred

In [ ]:
#4.4 : Misclassification rate
misclassification = np.mean(y_pred != y_test)
misclassification

In [ ]:
# 4.5 : Cost function
def average_cost(y_test, y_pred):
    """
    Input – y_test: vector with good/bad credit risk
          – y_pred: pred values for good/bad credit risk
    Output – average cost: constant
    """
    cost_vector = np.zeros_like(y_test)
    cost_vector[(y_pred == 0) & (y_test == 1)] = 5
    cost_vector[(y_pred == 1) & (y_test == 0)] = 1
    return np.mean(cost_vector)

avg_cost = average_cost(y_test, y_pred)
avg_cost

In [ ]:
adjusted_cost =adjust(y_test, y_pred)

avg_cost = average_cost(adjusted_cost)
avg_cost

In [ ]:
#4.6 : Maximum Likelihood Estimator




In [ ]:
#4.7 : Misclassification rate

In [ ]:
#4.8 : Prediction accuracy